<a href="https://colab.research.google.com/github/andrcvs/dio-lab-open-source/blob/main/notebooks/assistente_virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este projeto é um assistente virtual que utiliza reconhecimento de fala e síntese de voz para interagir com o usuário.

In [3]:
# ==============================================================================
# 1. INSTALAÇÃO DAS DEPENDÊNCIAS
# ==============================================================================
# ALTERADO: Trocamos 'wikipedia-api' por 'wikipedia'
print("Iniciando a instalação das dependências...")
!pip install SpeechRecognition gTTS wikipedia pydub # Added pydub
print("--- Instalação concluída com sucesso! ---")


# ==============================================================================
# 2. IMPORTAÇÕES E CONFIGURAÇÕES INICIAIS
# ==============================================================================
import speech_recognition as sr
from gtts import gTTS
from IPython.display import Audio, display, HTML
import wikipedia
import webbrowser
import os
import logging
from pydub import AudioSegment

# Configuração para evitar mensagens excessivas da biblioteca de áudio
logging.getLogger("pydub").setLevel(logging.WARNING)

# Configura o idioma diretamente na biblioteca wikipedia
wikipedia.set_lang('pt')
print("Bibliotecas carregadas e prontas para uso.")


# ==============================================================================
# 3. MÓDULO 1: TEXT-TO-SPEECH (SÍNTESE DE VOZ)
# ==============================================================================
def falar(texto):
    try:
        tts = gTTS(text=texto, lang='pt-br')
        tts.save("assistente_fala.mp3")
        display(Audio("assistente_fala.mp3", autoplay=True))
        print(f"Assistente: {texto}")
    except Exception as e:
        print(f"Ocorreu um erro na síntese de voz: {e}")


# ==============================================================================
# 4. MÓDULO 2: SPEECH-TO-TEXT (RECONHECIMENTO DE FALA)
# ==============================================================================
from google.colab import output
from base64 import b64decode
from io import BytesIO

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.target.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.start()
  await sleep(time)
  recorder.stop()
  stream.getAudioTracks().forEach(track => track.stop())
})
"""

def get_audio(duration=5):
    print(f"Ouvindo por {duration} segundos...")
    display(HTML(f"<script>{RECORD}</script>"))
    s = output.eval_js(f'record({duration*1000})')
    print("Gravação finalizada.")
    b = b64decode(s.split(',')[1])
    return b

def ouvir_comando():
    r = sr.Recognizer()
    audio_data = get_audio(5)
    audio_bytes = BytesIO(audio_data) # Use audio_bytes to avoid confusion

    try:
        # Convert audio data to WAV format using pydub
        audio_segment = AudioSegment.from_file(audio_bytes)
        audio_wav = BytesIO()
        audio_segment.export(audio_wav, format="wav")
        audio_wav.seek(0) # Rewind the BytesIO object

        with sr.AudioFile(audio_wav) as source:
            audio = r.record(source)
        texto_comando = r.recognize_google(audio, language='pt-BR')
        print(f"Você disse: {texto_comando}")
        return texto_comando.lower()
    except sr.UnknownValueError:
        print("Não foi possível entender o áudio.")
        return None
    except sr.RequestError as e:
        print(f"Erro no serviço de reconhecimento de voz; {e}")
        return None
    except Exception as e: # Catch other potential errors during conversion
        print(f"Erro durante o processamento do áudio: {e}")
        return None


# ==============================================================================
# 5. MÓDULO 3: FUNÇÕES DE AUTOMAÇÃO
# ==============================================================================
#  A função de pesquisa foi reescrita para usar a nova biblioteca 'wikipedia'
def pesquisar_wikipedia(termo):
    """
    Pesquisa um termo na Wikipedia e fala um resumo do resultado.
    """
    try:
        falar(f"Pesquisando por {termo} na Wikipedia, um momento.")
        # Pede um resumo com 3 sentenças. A biblioteca lida com a busca.
        resumo = wikipedia.summary(termo, sentences=3)
        falar(f"Aqui está o que encontrei:")
        falar(resumo)
    except wikipedia.exceptions.PageError:
        falar(f"Desculpe, não encontrei nenhum artigo para o termo {termo}.")
    except wikipedia.exceptions.DisambiguationError as e:
        falar(f"O termo {termo} é ambíguo. Por favor, seja mais específico. Algumas opções são: {e.options[:3]}")
    except Exception as e:
        falar("Desculpe, ocorreu um erro inesperado ao acessar a Wikipedia.")
        print(f"Erro na Wikipedia: {e}")

def abrir_youtube():
    falar("Abrindo o YouTube.")
    url = "https://www.youtube.com"
    display(HTML(f'<a href="{url}" target="_blank">Clique aqui para abrir o YouTube</a>'))

def encontrar_farmacia_proxima():
    falar("Para encontrar a farmácia mais próxima, clique no link que vou gerar.")
    url = "https://www.google.com/maps/search/farmácia"
    display(HTML(f'<a href="{url}" target="_blank">Clique aqui para ver farmácias próximas no Google Maps</a>'))

# ==============================================================================
# 6. LOOP PRINCIPAL DE EXECUÇÃO
# ==============================================================================
def iniciar_assistente():
    falar("Assistente virtual iniciado. Diga 'assistente' seguido de um comando.")
    while True:
        print("\nAguardando comando de ativação ('assistente')...")
        comando = ouvir_comando()
        if comando and 'assistente' in comando:
            falar("Sim? Qual comando deseja executar?")
            comando_acao = ouvir_comando()
            if comando_acao:
                if 'pesquisar na wikipédia por' in comando_acao:
                    termo = comando_acao.replace('pesquisar na wikipédia por', '').strip()
                    if termo:
                        pesquisar_wikipedia(termo)
                    else:
                        falar("Não entendi o que pesquisar. Tente novamente.")
                elif 'abrir o youtube' in comando_acao:
                    abrir_youtube()
                elif 'farmácia mais próxima' in comando_acao:
                    encontrar_farmacia_proxima()
                elif 'desligar' in comando_acao:
                    falar("Desligando o sistema. Até mais!")
                    break
                else:
                    falar("Desculpe, não reconheci este comando.")

Iniciando a instalação das dependências...
--- Instalação concluída com sucesso! ---
Bibliotecas carregadas e prontas para uso.


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [9]:
# CÉLULA DE TESTE
iniciar_assistente()

Assistente: Assistente virtual iniciado. Diga 'assistente' seguido de um comando.

Aguardando comando de ativação ('assistente')...
Ouvindo por 5 segundos...


Gravação finalizada.
Não foi possível entender o áudio.

Aguardando comando de ativação ('assistente')...
Ouvindo por 5 segundos...


Gravação finalizada.


Você disse: assistente


Assistente: Sim? Qual comando deseja executar?
Ouvindo por 5 segundos...


...

Assistente: Abrindo o YouTube.



Aguardando comando de ativação ('assistente')...
Ouvindo por 5 segundos...


Gravação finalizada.
Não foi possível entender o áudio.

Aguardando comando de ativação ('assistente')...
Ouvindo por 5 segundos...


Gravação finalizada.
Você disse: parar

Aguardando comando de ativação ('assistente')...
Ouvindo por 5 segundos...


KeyboardInterrupt: 